# Setup

On some systems, this cell needs to be run twice to get the correct settings of matplotlib.

In [ ]:
from rrk import *
from scipy import interpolate
import time

plt.rc("font", family="serif", size=16.)

## Energy Conservative Fourier Method for the KdV Equation

In [ ]:
N = 256
L = 80
xplot = np.linspace(0, L, N+1)
x = xplot[0:-1]
dx = x[1] - x[0]
xi = np.fft.fftfreq(N) * N * 2*np.pi / L
mu = 0.5*L
A = 2

def f(u):
    uhat = np.fft.fft(u)
    u2hat = np.fft.fft(u*u)
    
    return -(u * np.real(np.fft.ifft(1j*xi*uhat)) + np.real(np.fft.ifft(1j*xi*u2hat))) / 3. \
            - np.real(np.fft.ifft(-1j*xi*xi*xi*uhat))

def eta(w):
    return 0.5 * np.sum(w*w) * dx

def d_eta(w):
    return w * dx

def soliton(x, A):
    tmp = np.cosh(np.sqrt(3*A) / 6 * x)
    return A / (tmp * tmp)
    
def u_analytical(t, A, mu):
    c = A / 3
    x_m_ct = np.abs(np.fmod(x - c*t - x[0], L)) + x[0]
    return soliton(x_m_ct - mu, A)

def compute_and_plot(erk, dt, t_final, A, mu, method=None):
    u0 = u_analytical(0., A, mu)
    
    if method == None:
        time_tmp = time.time()
        tt, uu = relaxed_ERK(erk, dt, f, u0, t_final, relaxed=False, newdt=True)
        time_tmp = time.time() - time_tmp
        print("Baseline scheme:   %.2e s" % time_tmp)

        time_tmp = time.time()
        tt_r, uu_r = relaxed_ERK(erk, dt, f, u0, t_final, relaxed=True, newdt=True)
        time_tmp = time.time() - time_tmp
        print("Relaxation scheme: %.2e s" % time_tmp)
    else:
        time_tmp = time.time()
        tt, uu = method(erk, dt, f, eta, d_eta, u0, t_final,
                        relaxed=False, method="brentq", newdt=True)
        time_tmp = time.time() - time_tmp
        print("Baseline scheme:   %.2e s" % time_tmp)

        time_tmp = time.time()
        tt_r, uu_r = method(erk, dt, f, eta, d_eta, u0, t_final,
                            relaxed=True, method="brentq", newdt=True)
        time_tmp = time.time() - time_tmp
        print("Relaxation scheme: %.2e s" % time_tmp)

        time_tmp = time.time()
        tt_p, uu_p = projection_DIRK(erk, dt, f, eta, d_eta, u0, t_final)
        time_tmp = time.time() - time_tmp
        print("Projection scheme: %.2e s" % time_tmp)

    plt.close("all")
    fig_error = plt.figure()
    errors = [np.linalg.norm(uu[:,i] - u_analytical(tt[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt))]
    plt.plot(tt, errors, label="Baseline")
    errors_r = [np.linalg.norm(uu_r[:,i] - u_analytical(tt_r[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt_r))]
    plt.plot(tt_r, errors_r, label="Relaxation")
    errors_p = [np.linalg.norm(uu_p[:,i] - u_analytical(tt_p[i], A, mu)) * np.sqrt(dx) for i in np.arange(len(tt_p))]
    plt.plot(tt_p, errors_p, label="Orthogonal Projection")
    plt.xlabel("$t$"); plt.ylabel("Error"); plt.xlim(tt[1], tt[-1])
    plt.xscale("log")
    plt.yscale("log")
    plt.legend(loc="best")

    fig_eta = plt.figure()
    H = [eta(uu[:,i]) for i in range(uu.shape[1])]
    plt.plot(tt, H - H[0], label="Baseline")
    H_r = [eta(uu_r[:,i]) for i in range(uu_r.shape[1])]
    plt.plot(tt_r, H_r - H_r[0], label="Relaxation")
    H_p = [eta(uu_p[:,i]) for i in range(uu_p.shape[1])]
    plt.plot(tt_p, H_p - H_p[0], label="Orthogonal Projection")
    plt.xlabel("$t$"); plt.ylabel("Change of Energy"); plt.xlim(tt[0], tt[-1])
    plt.yscale("symlog", linthreshy=1.e-14)
    plt.legend(loc="best")

    fig_space = plt.figure()
    plt.plot(x, uu[:,-1], label="Baseline")
    plt.plot(x, uu_r[:,-1], label="Relaxation")
    plt.plot(x, uu_p[:,-1], label="Orthogonal Projection")
    plt.plot(x, u_analytical(t_final, A, mu), label="Analytical")
    plt.xlabel("$x$"); plt.ylabel("$u$"); plt.xlim(x[0], x[-1])
    plt.legend(loc="best")
    
    return tt, uu, tt_r, uu_r, tt_p, uu_p, fig_error, fig_eta, fig_space


erk = rk.loadRKM("SDIRK23"); dt = 0.5; t_final = 600; method=convex_relaxed_DIRK

tt, uu, tt_r, uu_r, tt_p, uu_p, fig_error, fig_eta, fig_space = compute_and_plot(erk, dt, t_final, A, mu, method)

In [ ]:
ax = fig_error.axes[0]
ax.plot(tt, 2.0e-4*tt, ":", color="gray")
ax.annotate(r"$\mathcal{O}(t)$", (2.0e2, 1.5e-2), color="gray")
ax.plot(tt, 1.5e-4*tt*tt, ":", color="gray")
ax.annotate(r"$\mathcal{O}(t^2)$", (3.0e1, 7.0e-1), color="gray")
ax.set_ylabel("Error")
leg = ax.get_legend()
if leg is not None:
    leg.remove()
fig_error.savefig("../figures/error_growth__KdV_SDIRK23_error.pdf", bbox_inches="tight")

ax = fig_eta.axes[0]
leg = ax.get_legend()
if leg is not None:
    leg.remove()
fig_eta.savefig("../figures/error_growth__KdV_SDIRK23_energy.pdf", bbox_inches="tight")

ax = fig_error.axes[0]
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/error_growth__KdV_SDIRK23_legend.pdf", bbox_inches="tight")

ax = fig_space.axes[0]
leg = ax.get_legend()
if leg is not None:
    leg.remove()
fig_space.savefig("../figures/error_growth__KdV_SDIRK23_solution.pdf", bbox_inches="tight")

ax = fig_space.axes[0]
plt.figure()
handles, labels = ax.get_legend_handles_labels()
plt.figlegend(handles, labels, loc="center", ncol=4)
plt.savefig("../figures/error_growth__KdV_SDIRK23_space_legend.pdf", bbox_inches="tight")

In [ ]:
dt_baseline = np.diff(tt)
print(len(dt_baseline))
print(np.mean(dt_baseline))
print(np.median(dt_baseline))

print()


plt.figure()
dt_relaxation = np.diff(tt_r)
plt.plot(tt_r[:-2], dt_relaxation[:-1], color="black")
plt.xlabel(r"$t$")
plt.ylabel(r"$\gamma \Delta t$")
plt.ticklabel_format(axis="y", style="sci", scilimits=(-1,1))
plt.savefig("../figures/error_growth__KdV_SDIRK23_gamma_dt.pdf", bbox_inches="tight")

print(len(dt_relaxation))
print(np.mean(dt_relaxation))
print(np.median(dt_relaxation))

In [ ]:
mass = [sum(uu[:,i])*dx for i in range(uu.shape[1])]
mass_p = [sum(uu_p[:,i])*dx for i in range(uu_p.shape[1])]
mass_r = [sum(uu_r[:,i])*dx for i in range(uu_r.shape[1])]
plt.plot(tt,mass,label='Baseline')
plt.plot(tt_r,mass_r,label='Relaxation')
plt.plot(tt_p,mass_p,label='Orthogonal Projection')
plt.ylabel('Mass')
plt.xlabel('$t$')
plt.xlim(tt[0], tt[-1])
plt.savefig('../figures/error_growth__KdV_SDIRK23_mass.pdf', bbox_inches="tight")